<a href="https://colab.research.google.com/github/ahmabboud/FellowAI/blob/main/VideoExtractor_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Use opencv to work on video stream from Youtube


This will Etract videos from Youtube to build Database

In [ ]:
!apt-get install python3-OpenCV
!pip install pafy
!pip install youtube_dl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package python3-OpenCV
     |████████████████████████████████| 1.8MB 3.2MB/s 


In [ ]:
import cv2
import matplotlib
import numpy as np
import pafy
from google.colab.patches import cv2_imshow
import time
import uuid
import matplotlib.pyplot as plt
# Python program to show time by process_time()  
from time import process_time

In [ ]:

#Check if 2 images are equals
def isDuplicat(original,Unique):
  for t in Unique:
      if original.shape == t.shape:
        difference = cv2.subtract(original, t)
        b, g, r = cv2.split(difference)
        if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
          print("find Duplicate")
          return True
        else:
          return False
      else:
        return False


In [ ]:
import multiprocessing
from multiprocessing.pool import ThreadPool as Pool
# from multiprocessing import Pool

In [ ]:
from shapely.geometry import Polygon
def overlap2(rect1,rect2):
    comparison = rect1 == rect2
    equal_arrays = comparison.all()
    if equal_arrays:
      return True
    else:
      p1 = Polygon([(rect1[0],rect1[1]), (rect1[0]-rect1[2],rect1[1]),(rect1[0]-rect1[2],rect1[1]-rect1[3]),(rect1[0],rect1[1]-rect1[3])])
      p2 = Polygon([(rect2[0],rect2[1]), (rect2[0]-rect2[2],rect2[1]),(rect2[0]-rect2[2],rect2[1]-rect2[3]),(rect2[0],rect2[1]-rect2[3])])
      #x,y = p1.exterior.xy
      #plt.plot(x,y)
      #x,y = p2.exterior.xy
      #plt.plot(x,y)
      #print(p1.intersects(p2)) 
      return (p1.intersects(p2))
#[855 512  50  50] [616  46  55  55]
overlap2(np.array([855, 512,  50,  50,]),np.array([616,  46,  55,  55]))

False

In [ ]:
def worker(item,capture,path,maxi,FrameSkipR,f):
    global Uid
    try:
        #api.my_operation(item)
        created = multiprocessing.Process()
        current = multiprocessing.current_process()
        #print ('running:', current.name, current._identity)
        #print ('created:', created.name, created._identity)
        (x,y,w,h)=item
        
        FW2ImgRt= int(((1-0.08))*w) # considered 10% face width to image width
        FH2ImgRt=int(((1-0.3))*h) # considered 30% face height to image height
        i=0
        Nbrf=0
        while Nbrf<=maxi and capture.isOpened:
          ret, img = capture.read()
          
          if i%(FrameSkipR)==0:
            body=img[y-int((FH2ImgRt)*0.15):y+(h+int((FH2ImgRt)*0.3)), (x-int(FW2ImgRt/2)):x+(w+int(FW2ImgRt/2))]
            
            cv2.imwrite(path+'V'+Uid+'_'+str(f)+'_'+str(i)+'.jpg',body)
            Nbrf+=1

          i+=1
    except Exception as e:
      print(e)


In [ ]:
 # Start the stopwatch / counter  
t1_start = process_time() 

file1 = open('/content/drive/My Drive/FellowAI/Dataset_Emo/ZoomLinks.txt', 'r') # file containing YouTube zoom links
Lines = file1.readlines() 
  
# Strips the newline character 
for line in Lines:     
    
    face_cascade = cv2.CascadeClassifier('/content/drive/My Drive/FellowAI/temp/haarcascade_frontalface_default.xml') # Haar Cascade File
    url=line.strip()
    print(url)
    Uid=url[-11:]       # Ue VideoID as UID
    print(Uid)
    
    video = pafy.new(url)
    best = video.getbest()    # get best resolution
    capture = cv2.VideoCapture()
    capture.open(best.url)
    fps = int(capture.get(cv2.CAP_PROP_FPS)) # Frame Per sec
    print("fps,",fps)


    ''' face detection in Video
    '''
    NbrFaces=10         # Minimum number of faces
    #path='/content/drive/My Drive/FellowAI/dataset/Ahmad_Draft/'
    path='/content/drive/My Drive/FellowAI/Dataset_Emo/Draft3/' # Path to the Destination DS file 
    Nbrframs=10000        # Maximum number of frames to search
    FrameSkip=2     #seconds skips capture frame each # sec
    StartFrame=fps*10   # start capture after 10 sec
    capture.set(cv2.CAP_PROP_POS_FRAMES, StartFrame)
    i=0
    while capture.isOpened and i<=Nbrframs:

      ret, img = capture.read()
      #print('frame',i)
      if i%(fps)==0:      # frame each sec
          gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
          faces = face_cascade.detectMultiScale(gray, 1.1, 5,minSize = (10, 10)) 
          #print('Faces:',len(faces))
         # delete overlaped faces and keep separated
          SepFaces=[]
          c=0
          for face in faces:
            if len(SepFaces)==0:
              SepFaces.append(np.array(face))
            else:
              c=1
              for s,Sep in enumerate(SepFaces):
                #print(s,face,Sep)
                if not overlap2(face,Sep):
                  c+=1
                else:
                  #print("Face overlap",face,Sep)
                  break
              if c>= len(SepFaces):     
                SepFaces.append(np.array(face))
                #print('face appended',len(SepFaces))

          print('face appended',len(SepFaces))  
          if len(SepFaces)>=4:      # run just if number of detected faces is >= 4
            pool = Pool(len(SepFaces))
            
            for f,face in enumerate(SepFaces):
                # foreach face detected apply a thread fro the pool of threads
                pool.apply(worker, args=(face,capture,path,NbrFaces,fps*FrameSkip,f))
            pool.close()
            pool.join()
            break
      i+=1
    capture.release()
    cv2.destroyAllWindows()
file1.close()
# Stop the stopwatch / counter 
t1_stop = process_time() 
print("Elapsed time:", t1_stop, t1_start)   
print("Elapsed time during the whole program in seconds:", t1_stop - t1_start)

https://www.youtube.com/watch?v=lVtEB_QnZk0
lVtEB_QnZk0
fps, 25
face appended 2
face appended 2
face appended 7
face appended 9
face appended 11
face appended 12
Elapsed time: 1352.031043619 1288.69943249
Elapsed time during the whole program in seconds: 63.331611129000066
